# 9. Dash analysis

Run `python3 09-dashboard.py` fro your command line to start the dash server.

This Notebook brings some numbers in the form of tables to answer the following questions.

1. Launch site with a largest success launch rate and number of successes: **KSC LC-39A**
2. Launch site with a lowest success launch rate and number of failures: **CCAFS LC-40 76.92%**
3. Payload range with highest success rate: **2000 - 5000 kg**
4. Payload range with lowest success rate: **(0-2000 kg) & (5000-7000 kg)**
5. Booster with highest number of launch success: **F9 FT with 66.69%**


## Imports


In [3]:
import pandas as pd
from pathlib import Path

import helpers as hlp  # helper functions

## Setup


In [4]:
INPUT_FILE = hlp.DATA_DIR / Path("spacex_launch_dash.csv")
PAYLOAD_BINS = list(range(0, 10001, 1000))

## Load data


In [5]:
df = pd.read_csv(INPUT_FILE, encoding="utf-8", header=0, index_col=False)

# normalize column names
df.columns = [hlp.normalize_column_name(col) for col in df.columns]
df = df.drop("unnamed:_0", axis=1)
df.head()

,flight_number,launch_site,class,payload_mass_kg,booster_version,booster_version_category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


## Common code


In [6]:
unique_sites = list(df["launch_site"].unique())
unique_booster_versions = df["booster_version"].apply(lambda x: x.split()[1]).unique()

## 1. Launch site with largest / lowest success launch rate


In [13]:
# intermediary lists
launches = []
successes = []
failures = []

# populate intermediary lists
for booster_version in unique_sites:
    launches.append(df.loc[(df["launch_site"] == booster_version)]["class"].count())
    successes.append(
        df.loc[(df["launch_site"] == booster_version) & (df["class"] == 1)][
            "class"
        ].count()
    )
    failures.append(
        df.loc[(df["launch_site"] == booster_version) & (df["class"] == 0)][
            "class"
        ].count()
    )

# create a new DataFrame to host launches, succeeded and failed launches
success_df = pd.DataFrame(
    data={
        "site": unique_sites,
        "launches": launches,
        "success": successes,
        "failure": failures,
    }
)
success_df["success_ratio"] = round(success_df["success"] / success_df["launches"], 4)
success_df["failure_ratio"] = round(success_df["failure"] / success_df["launches"], 4)
success_df

,site,launches,success,failure,success_ratio,failure_ratio
0,CCAFS LC-40,26,7,19,0.2692,0.7308
1,VAFB SLC-4E,10,4,6,0.4000,0.6000
2,KSC LC-39A,13,10,3,0.7692,0.2308
3,CCAFS SLC-40,7,3,4,0.4286,0.5714


## 3. Payload success / failure rate


In [8]:
intervals_df = (
    pd.cut(df["payload_mass_kg"], PAYLOAD_BINS, right=True).value_counts().sort_index()
)
intervals_df = intervals_df.reset_index()
intervals_df

,payload_mass_kg,count
0,"(0, 1000]",8
1,"(1000, 2000]",3
2,"(2000, 3000]",10
3,"(3000, 4000]",11
4,"(4000, 5000]",8
5,"(5000, 6000]",5
6,"(6000, 7000]",4
7,"(7000, 8000]",0
8,"(8000, 9000]",0
9,"(9000, 10000]",5


In [9]:
# intermediary lists
unique_intervals = []
launches = []
successes = []
failures = []

# populate intermediary lists
for row in intervals_df.itertuples():
    _, interval, counts = row
    min_, max_ = [interval.left, interval.right]
    unique_intervals.append([min_, max_])

    payload_condition = (min_ <= df["payload_mass_kg"]) & (df["payload_mass_kg"] < max_)
    launches.append(df.loc[payload_condition]["class"].count())
    successes.append(df.loc[(payload_condition) & (df["class"] == 1)]["class"].count())
    failures.append(df.loc[(payload_condition) & (df["class"] == 0)]["class"].count())

# create a new DataFrame to host intervals, succeeded and failed launches
success_df = pd.DataFrame(
    data={
        "interval": unique_intervals,
        "launches": launches,
        "success": successes,
        "failure": failures,
    }
)
success_df["success_ratio"] = round(success_df["success"] / success_df["launches"], 4)
success_df["failure_ratio"] = round(success_df["failure"] / success_df["launches"], 4)
success_df = success_df.fillna(0)  # set NaNs to 0
success_df

,interval,launches,success,failure,success_ratio,failure_ratio
0,"[0, 1000]",10,2,8,0.2000,0.8000
1,"[1000, 2000]",3,1,2,0.3333,0.6667
2,"[2000, 3000]",10,5,5,0.5000,0.5000
3,"[3000, 4000]",11,8,3,0.7273,0.2727
4,"[4000, 5000]",8,3,5,0.3750,0.6250
5,"[5000, 6000]",5,2,3,0.4000,0.6000
6,"[6000, 7000]",4,0,4,0.0000,1.0000
7,"[7000, 8000]",0,0,0,0.0000,0.0000
8,"[8000, 9000]",0,0,0,0.0000,0.0000
9,"[9000, 10000]",5,3,2,0.6000,0.4000


## 5. Booster with highest number of launches


In [10]:
# add new column to host booster versions
df["version"] = df["booster_version"].apply(lambda x: x.split()[1])

In [11]:
# intermediary lists
launches = []
successes = []
failures = []

# populate intermediary lists
for booster_version in unique_booster_versions:
    launches.append(df.loc[(df["version"] == booster_version)]["class"].count())
    successes.append(
        df.loc[(df["version"] == booster_version) & (df["class"] == 1)]["class"].count()
    )
    failures.append(
        df.loc[(df["version"] == booster_version) & (df["class"] == 0)]["class"].count()
    )

# create a new DataFrame to host booster versions, succeeded and failed launches
success_df = pd.DataFrame(
    data={
        "booster": unique_booster_versions,
        "launches": launches,
        "success": successes,
        "failure": failures,
    }
)
success_df["success_ratio"] = round(success_df["success"] / success_df["launches"], 4)
success_df["failure_ratio"] = round(success_df["failure"] / success_df["launches"], 4)
success_df

,booster,launches,success,failure,success_ratio,failure_ratio
0,v1.0,5,0,5,0.0000,1.0000
1,v1.1,15,1,14,0.0667,0.9333
2,FT,24,16,8,0.6667,0.3333
3,B4,11,6,5,0.5455,0.4545
4,B5,1,1,0,1.0000,0.0000
